In [1]:
from modules.loader.DataLoader import DataLoader
from modules.processor.DataProcessor import DataProcessor
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
from sksurv.metrics import concordance_index_ipcw
from sksurv.util import Surv
from sklearn.model_selection import KFold
from scipy.stats import uniform, randint
import random
from typing import cast

In [2]:
num_features = ['age', 'gender', 'BMI', 'DIA_MAX_TUMEUR_1', 'DIA_MAX_TUMEUR_2', 'DIA_MAX_TUMEUR_3', 'DIA_MAX_TUMEUR_4',
              'INSUFFURENALE_ON', 'FUMEUROUINON', 'DIABETIQUE_ON', 'INSULO_ON', 'PACEMAKER_ON', 'ALLERGIE_ON',
              'TotalDose', 'NbVolumes', 'TotalSessions', 'DEGRURG', 'CHIMOANT_ON', 'IRRA_ANT_ON', 'CHIRURGIE_ANT_ON']

cat_ohe_features = ['ICD9', 'IRRANT_OLOCA', 'MOBPAT', 'T', 'N', 'M']

cat_ord_features = ['NRSF11', 'OMS']

In [3]:
data_loader = DataLoader()
df_raw = data_loader.load_from_csv()
data_processor = DataProcessor(num_features, cat_ohe_features, cat_ord_features)
df, df_filtered = data_processor.get_df(df_raw)

Subset length: 536
Subset shape: (536, 50)


In [28]:
features = num_features + cat_ohe_features + cat_ord_features
X = df[features]

Y_time  = df['time_in_months']
Y_event = df['dead']

In [5]:
all_indices = df.index.to_numpy()
train_indices, test_indices = train_test_split(
    all_indices,
    test_size=0.2,
    random_state=42
)
X_train = X.loc[train_indices]
X_test = X.loc[test_indices]
time_train = Y_time.loc[train_indices]
time_test = Y_time.loc[test_indices]
event_train = Y_event.loc[train_indices]
event_test = Y_event.loc[test_indices]

y_train = Surv.from_arrays(event_train, time_train)
y_test = Surv.from_arrays(event_test, time_test)

In [6]:
ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=999)
ohe_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
preprocessor = ColumnTransformer(
    [
        ("numerical", StandardScaler(), num_features),
        ("cat_ord", ord_encoder, cat_ord_features),
        ("cat_ohe", ohe_encoder, cat_ohe_features)
    ],
    verbose_feature_names_out=False
)
preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                 ['age', 'gender', 'BMI', 'DIA_MAX_TUMEUR_1',
                                  'DIA_MAX_TUMEUR_2', 'DIA_MAX_TUMEUR_3',
                                  'DIA_MAX_TUMEUR_4', 'INSUFFURENALE_ON',
                                  'FUMEUROUINON', 'DIABETIQUE_ON', 'INSULO_ON',
                                  'PACEMAKER_ON', 'ALLERGIE_ON', 'TotalDose',
                                  'NbVolumes', 'TotalSessions', 'DEGRURG',
                                  'CHIMOANT_ON', 'IRRA_ANT_ON',
                                  'CHIRURGIE_ANT_ON']),
                                ('cat_ord',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=999),
                                 ['NRSF11', 'OMS']),
                                ('cat_ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['ICD9', 'IRRANT_OLOCA', 'MOBPAT', 'T', 'N',
                                  'M'])],
                  verbose_feature_names_out=False)

In [7]:
X_train = cast(pd.DataFrame, preprocessor.fit_transform(X_train))
X_test = cast(pd.DataFrame, preprocessor.transform(X_test))

In [8]:
# --- 3. ФУНКЦІЯ СТВОРЕННЯ DMatrix З МЕЖАМИ ЦЕНЗУРУВАННЯ ---
def create_dmatrix(X, time, event):
    dmat = xgb.DMatrix(X, label=time)
    dmat.set_float_info('label_lower_bound', time)
    dmat.set_float_info('label_upper_bound', np.where(event==1, time, np.inf))
    return dmat

In [9]:
params = {'objective': 'survival:aft',
               'aft_loss_distribution_scale': 1.0,
               'aft_loss_distribution': 'logistic'}

In [10]:
dtrain = create_dmatrix(X_train, time_train, event_train)
dtest = create_dmatrix(X_test, time_test, event_test)
bst = xgb.train(params, dtrain, num_boost_round=30000,
            evals=[(dtest, 'test')],
            early_stopping_rounds=500,
            verbose_eval=500)

[0]	test-aft-nloglik:2.52737
[500]	test-aft-nloglik:2.88977
[504]	test-aft-nloglik:2.89173


In [11]:
months = bst.predict(dtest)

In [12]:
print(f"XGBoost c-index: {concordance_index(time_test, months, event_test)}")
print(f"IPCW C-index: {concordance_index_ipcw(y_train, y_test, -months)[0]:.4f}")
# - monotone_constraints
# XGBoost c-index: 0.73446
# + monotone_constraints
# XGBoost c-index: 0.7251908396946565

XGBoost c-index: 0.6336254107338445
IPCW C-index: 0.5333


In [13]:
from lifelines import KaplanMeierFitter
def calculate_ipcw_mse_lifelines(y_actual, y_predicted):
    event_status = y_actual['event']
    actual_time = y_actual['time']

    # 1. Оцінювання функції цензурування G(t) за допомогою lifelines
    # Для G(t) ми моделюємо "виживання" ЦЕНЗУРУВАННЯ.
    # Статус події має бути інвертований: (1 - event_status)
    kmf_censoring = KaplanMeierFitter()
    kmf_censoring.fit(durations=actual_time, event_observed=~event_status) # ~event_status інвертує True/False

    # 2. Отримання оцінок G(t) для кожного фактичного часу
    # kmf_censoring.predict() повертає оцінки виживаності для заданих часів
    G_t = kmf_censoring.predict(actual_time)

    # 3. Запобігання діленню на нуль (забезпечуємо мінімальне значення)
    G_t[G_t < 1e-10] = 1e-10

    # 4. Обчислюємо IPCW-ваги: W_i = I(event) / G(T_i)
    # Зверніть увагу: event_status повинен бути типу float для коректного ділення
    ipcw_weights = event_status.astype(float) / G_t

    # 5. Обчислюємо квадрат помилки
    sq_error = (actual_time - y_predicted)**2

    # 6. Обчислюємо зважену MSE
    # Ділимо на загальну кількість нецензурованих пацієнтів
    weighted_mse = np.sum(ipcw_weights * sq_error) / np.sum(event_status)

    return weighted_mse

In [14]:
def cross_validate_xgboost_params(params, X, time, event, n_splits=5):
  # Крос-валідація KFold
  kf = KFold(n_splits, shuffle=True, random_state=42)
  cv_scores = []
  cv_mse = []
  for fold, (train_index, test_index) in enumerate(kf.split(X)):

      # Розділення даних
      # Припускаємо, що X_train, time_train, event_train є об'єктами з індексацією
      X_fold_train, X_fold_test = X.iloc[train_index], X.iloc[test_index]

      X_fold_train = cast(pd.DataFrame, preprocessor.fit_transform(X_fold_train))
      X_fold_test = cast(pd.DataFrame, preprocessor.transform(X_fold_test))

      time_fold_train, time_fold_test = time.iloc[train_index], time.iloc[test_index]
      event_fold_train, event_fold_test = event.iloc[train_index], event.iloc[test_index]

      y_actual = Surv.from_arrays(event=event_fold_test, time=time_fold_test)

      # Створення DMatrix
      dtrain = create_dmatrix(X_fold_train, time_fold_train, event_fold_train)
      dtest = create_dmatrix(X_fold_test, time_fold_test, event_fold_test)

      # Навчання моделі
      bst = xgb.train(
          params,
          dtrain,
          num_boost_round=30000,
          evals=[(dtest, 'test')],
          early_stopping_rounds=500,
          verbose_eval=False
      )
      # Прогноз часу
      months_pred = bst.predict(dtest)

      ipcw_mse_value = calculate_ipcw_mse_lifelines(y_actual, months_pred)

      # Обчислення C-індексу (lifelines)
      c_index = concordance_index(time_fold_test, months_pred, event_fold_test)
      # print(f"  Фолд {fold+1}/5: C-індекс = {c_index:.4f}")
      cv_scores.append(c_index)
      cv_mse.append(ipcw_mse_value)


  mean_cv_score = np.mean(cv_scores)
  mean_cv_mse = np.mean(cv_mse)
  print(f"Середній C-індекс по {n_splits} фолдам: {mean_cv_score:.4f}")
  print(f"Середній MSE по {n_splits} фолдам: {mean_cv_mse:.4f}")
  return mean_cv_score, mean_cv_mse

In [15]:
cross_validate_xgboost_params(params, X, Y_time, Y_event)
# - monotone_constraints
# X
# Середній C-індекс по 5 фолдам: 0.6740
# + monotone_constraints
# X
# Середній C-індекс по 5 фолдам: 0.6717

Середній C-індекс по 5 фолдам: 0.6490
Середній MSE по 5 фолдам: 237297823.5945


(np.float64(0.6489583236171281), np.float64(237297823.59449258))

In [ ]:
# --- 1. ПРИПУЩЕННЯ: Ви вже маєте X_train, time_train, event_train (як ndarray або Series/DataFrame) ---

# --- 2. ПРОСТІР ПАРАМЕТРІВ ДЛЯ ПОШУКУ ---
# Розширюємо діапазони навколо ваших поточних параметрів для L1/L2 та глибини
param_dist = {
    # Регуляризація
    'reg_alpha': uniform(0.0, 3.0),      # L1 (Lasso)
    'reg_lambda': uniform(0.5, 5.0),     # L2 (Ridge)
    'min_child_weight': randint(5, 30),  # Дозволяємо менші значення

    # Параметри дерева
    'max_depth': randint(3, 6),          # Глибина [3, 4, 5]

    # Параметри навчання
    'learning_rate': [0.001, 0.005, 0.01, 0.02], # Крок навчання
    'subsample': uniform(0.7, 0.3),      # Частка зразків [0.7 - 1.0]
    'colsample_bytree': uniform(0.7, 0.3),# Частка ознак [0.7 - 1.0]

    # AFT Розподіл (спробуємо різні)
    'aft_loss_distribution': ['extreme', 'normal', 'logistic'],
    'aft_loss_distribution_scale': uniform(0.5, 1.2)
}

# Фіксовані параметри
base_params = {
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'tree_method': 'auto',
    'seed': 42,
    #'aft_loss_distribution_scale': 0.79, # Залишаємо ваше фіксоване значення
    # 'monotone_constraints': tuple(monotone),
    'verbosity': 0
}

In [19]:
def cross_validate_xgboost(base_params, param_dist, X, time, event, n_splits=5, n_iter = 100):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    best_score = 0.0
    best_mse = np.inf
    best_params = None

    print(f"Початок Randomized Search на {n_iter} ітерацій...")

    # 1. Створення випадкових комбінацій параметрів
    random_combinations = []
    for _ in range(n_iter):
        params = base_params.copy()
        for k, v in param_dist.items():
            if isinstance(v, list):
                params[k] = random.choice(v)
            else:
                params[k] = v.rvs(1)[0] # Генерація випадкового числа
        random_combinations.append(params)

    # 2. Перебір випадкових комбінацій
    for i, current_params in enumerate(random_combinations):

        mean_cv_score, mean_cv_mse = cross_validate_xgboost_params(current_params, X, time, event)
        print(f"Ітерація {i+1}/{n_iter}: Кращий індекс: {best_mse}, C-індекс = {mean_cv_score:.4f}, Кращі параметри: {best_params}")

        # Оновлення найкращого результату
        # if mean_cv_score > best_score:
        #     best_score = mean_cv_score
        #     best_params = current_params.copy()
        if mean_cv_mse < best_mse:
            best_mse = mean_cv_mse
            best_params = current_params.copy()

    print("\n--- ПОШУК ЗАВЕРШЕНО ---")
    # print(f"Найкращий C-індекс крос-валідації: {best_score:.4f}")
    print(f"Найкращbq mse крос-валідації: {best_mse:.4f}")
    print(f"Найкращі параметри: {best_params}")
    return best_params, best_score

In [20]:
best_params, best_score = cross_validate_xgboost(base_params, param_dist, X, Y_time, Y_event, 5, 1000)
# cross_validate_xgboost_params(best_params, X, Y_time, Y_event)

# - monotone_constraints, Randomezed Search
# X_test
# Найкращий C-індекс крос-валідації: 0.6877
# Найкращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'aft_loss_distribution_scale': 0.79, 'verbosity': 0, 'reg_alpha': np.float64(2.1819391982742613), 'reg_lambda': np.float64(2.748985534798474), 'min_child_weight': np.int64(5), 'max_depth': np.int64(4), 'learning_rate': 0.001, 'subsample': np.float64(0.700298421652543), 'colsample_bytree': np.float64(0.8617271824257702), 'aft_loss_distribution': 'logistic'}
# aft_loss_distribution_scale вільний
# Найкращий C-індекс крос-валідації: 0.6867
# Найкращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'verbosity': 0, 'reg_alpha': np.float64(1.9519820605754257), 'reg_lambda': np.float64(0.9478336482171974), 'min_child_weight': np.int64(8), 'max_depth': np.int64(4), 'learning_rate': 0.001, 'subsample': np.float64(0.7355526548236677), 'colsample_bytree': np.float64(0.7791497167769112), 'aft_loss_distribution': 'logistic', 'aft_loss_distribution_scale': np.float64(0.6810410426323612)}
# X
# Середній C-індекс по 5 фолдам: 0.6871
# aft_loss_distribution_scale вільний
# Середній C-індекс по 5 фолдам: 0.6869
# + monotone_constraints, Randomezed Search
# X_test
# Найкращий C-індекс крос-валідації: 0.6863
# Найкращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'aft_loss_distribution_scale': 0.79, 'monotone_constraints': (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'verbosity': 0, 'reg_alpha': np.float64(2.991818627243018), 'reg_lambda': np.float64(4.353225143660476), 'min_child_weight': np.int64(14), 'max_depth': np.int64(3), 'learning_rate': 0.001, 'subsample': np.float64(0.8297631851161293), 'colsample_bytree': np.float64(0.7079318036224579), 'aft_loss_distribution': 'logistic'}
# X
# Середній C-індекс по 5 фолдам: 0.6861


Початок Randomized Search на 1000 ітерацій...
Середній C-індекс по 5 фолдам: 0.6896
Середній MSE по 5 фолдам: 2163.8171
Ітерація 1/1000: Кращий індекс: inf, C-індекс = 0.6896, Кращі параметри: None
Середній C-індекс по 5 фолдам: 0.6997
Середній MSE по 5 фолдам: 1032.7032
Ітерація 2/1000: Кращий індекс: 2163.8171009096504, C-індекс = 0.6997, Кращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'verbosity': 0, 'reg_alpha': np.float64(1.2298175498747637), 'reg_lambda': np.float64(1.7843269010360114), 'min_child_weight': np.int64(10), 'max_depth': np.int64(4), 'learning_rate': 0.02, 'subsample': np.float64(0.9189677108472178), 'colsample_bytree': np.float64(0.9601744747859046), 'aft_loss_distribution': 'logistic', 'aft_loss_distribution_scale': np.float64(0.7204456715951443)}
Середній C-індекс по 5 фолдам: 0.6783
Середній MSE по 5 фолдам: 547.6559
Ітерація 3/1000: Кращий індекс: 1032.7032176210523, C-індекс = 0.6783, Кращі параметр

In [22]:
# -N, M
# Найкращий C-індекс крос-валідації: 0.7034
# Найкращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'verbosity': 0, 'reg_alpha': np.float64(0.6142334687110949), 'reg_lambda': np.float64(4.028982296602884), 'min_child_weight': np.int64(5), 'max_depth': np.int64(3), 'learning_rate': 0.001, 'subsample': np.float64(0.7354002652169873), 'colsample_bytree': np.float64(0.9584650499583631), 'aft_loss_distribution': 'logistic', 'aft_loss_distribution_scale': np.float64(0.5356948453218708)}
# Середній C-індекс по 5 фолдам: 0.6906
# + N, M
# Найкращий C-індекс крос-валідації: 0.6999
# Найкращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'verbosity': 0, 'reg_alpha': np.float64(1.885273386835653), 'reg_lambda': np.float64(1.3866093699752953), 'min_child_weight': np.int64(11), 'max_depth': np.int64(3), 'learning_rate': 0.001, 'subsample': np.float64(0.8526048126701572), 'colsample_bytree': np.float64(0.7348574730865784), 'aft_loss_distribution': 'logistic', 'aft_loss_distribution_scale': np.float64(0.5219336218198749)}
# Середній C-індекс по 5 фолдам: 0.6913

# X
# Найкращий C-індекс крос-валідації: 0.7100214916839873
# Найкращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'verbosity': 0, 'reg_alpha': np.float64(0.5052099414307786), 'reg_lambda': np.float64(0.6377107884457474), 'min_child_weight': np.int64(5), 'max_depth': np.int64(4), 'learning_rate': 0.001, 'subsample': np.float64(0.8669538641818011), 'colsample_bytree': np.float64(0.8951288066582845), 'aft_loss_distribution': 'logistic', 'aft_loss_distribution_scale': np.float64(0.9914290128527132)}
# Найкращий mse крос-валідації: 293.7952
# Найкращі параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'verbosity': 0, 'reg_alpha': np.float64(0.8047482880588891), 'reg_lambda': np.float64(5.053721686151811), 'min_child_weight': np.int64(14), 'max_depth': np.int64(4), 'learning_rate': 0.001, 'subsample': np.float64(0.9556029688506168), 'colsample_bytree': np.float64(0.7086830419232711), 'aft_loss_distribution': 'normal', 'aft_loss_distribution_scale': np.float64(0.5932114633181479)}

best_params = {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'verbosity': 0, 'reg_alpha': np.float64(0.8047482880588891), 'reg_lambda': np.float64(5.053721686151811), 'min_child_weight': np.int64(14), 'max_depth': np.int64(4), 'learning_rate': 0.001, 'subsample': np.float64(0.9556029688506168), 'colsample_bytree': np.float64(0.7086830419232711), 'aft_loss_distribution': 'normal', 'aft_loss_distribution_scale': np.float64(0.5932114633181479)}

dtrain = create_dmatrix(X_train, time_train, event_train)
dtest = create_dmatrix(X_test, time_test, event_test)
bst = xgb.train(best_params, dtrain, num_boost_round=30000,
            evals=[(dtest, 'test')],
            early_stopping_rounds=500,
            verbose_eval=500)
print("Best score: ", bst.best_score)
num_rounds = bst.best_iteration
print("Best iteration: ", bst.best_iteration)

times_preds = bst.predict(dtest)
print(f"Final XGBoost c-index: {concordance_index(time_test, times_preds, event_test)}")
print(f"Final XGBoost mse: {calculate_ipcw_mse_lifelines(y_test, times_preds)}")

result = pd.DataFrame({'xgb_aft_time': times_preds, 'true_time' : df.loc[test_indices, 'time_in_months'], 'ES_VIM': df.loc[test_indices, 'ES_VIM'], 'dead': event_test})
result = result[result['dead'] == 1].reset_index(drop=True)
print(result)

print(result[['xgb_aft_time', 'true_time']].describe())
print(result[['xgb_aft_time', 'true_time']].quantile([0.1, 0.25, 0.5, 0.75, 0.9]))

[0]	test-aft-nloglik:18.33509
[500]	test-aft-nloglik:9.58327
[1000]	test-aft-nloglik:5.65881
[1500]	test-aft-nloglik:4.10422
[2000]	test-aft-nloglik:3.46116
[2500]	test-aft-nloglik:3.20241
[3000]	test-aft-nloglik:3.09846
[3500]	test-aft-nloglik:3.06226
[4000]	test-aft-nloglik:3.05010
[4500]	test-aft-nloglik:3.04580
[4904]	test-aft-nloglik:3.04512
Best score:  3.0437909201623863
Best iteration:  4404
Final XGBoost c-index: 0.7127601314348302
Final XGBoost mse: 374.51223426068617
    xgb_aft_time  true_time  ES_VIM  dead
0      16.888391         12    18.0     1
1      11.890463         15     NaN     1
2      30.895180         41     NaN     1
3       9.387396          8     NaN     1
4      16.364237         22     6.0     1
5      28.457129         39    48.0     1
6       3.314278          3    24.0     1
7      23.924383         17    24.0     1
8      14.711249         30    30.0     1
9      14.086128          1     NaN     1
10     35.808861         34     NaN     1
11      9.046

In [29]:
X = cast(pd.DataFrame, preprocessor.fit_transform(X))
dtrain = create_dmatrix(X, Y_time, Y_event)

# df_filtered = df_filtered[df_filtered['dead'] == 1]
df_filtered['time_in_months'] = df_filtered['time_in_months'].fillna(999)
print(f'Validation size: {df_filtered.shape[0]}')

X_val = df_filtered[num_features + cat_ohe_features + cat_ord_features]
X_val = cast(pd.DataFrame, preprocessor.transform(X_val))

Y_time_val  = df_filtered['time_in_months']
Y_event_val = df_filtered['dead']

dval = create_dmatrix(X_val, Y_time_val, Y_event_val)

bst = xgb.train(best_params, dtrain, num_boost_round=num_rounds + 1,
            evals=[(dval, 'test')],
            early_stopping_rounds=500,
            verbose_eval=500)
# print("Best score: ", bst.best_score)
# print("Best iteration: ", bst.best_iteration)

times_preds = bst.predict(dval)
print(f"Total XGBoost c-index: {concordance_index(Y_time_val, times_preds, Y_event_val)}")
y = Surv.from_arrays(Y_event_val, Y_time_val)
print(f"Total XGBoost mse: {calculate_ipcw_mse_lifelines(y, times_preds)}")

result = pd.DataFrame({'xgb_aft_time': times_preds, 'true_time' : Y_time_val, 'ES_VIM': df_filtered['ES_VIM'], 'dead': Y_event_val})
result = result[result['dead'] == 1].reset_index(drop=True)
print(result)

Validation size: 28
[0]	test-aft-nloglik:24.51089
[500]	test-aft-nloglik:22.96592
[1000]	test-aft-nloglik:22.50542
[1500]	test-aft-nloglik:22.38966
[2000]	test-aft-nloglik:21.96945
[2500]	test-aft-nloglik:20.91916
[3000]	test-aft-nloglik:19.95642
[3500]	test-aft-nloglik:19.26007
[4000]	test-aft-nloglik:18.78828
[4404]	test-aft-nloglik:18.50594
Total XGBoost c-index: 0.7931034482758621
Total XGBoost mse: 86.79264972231647
   xgb_aft_time  true_time  ES_VIM  dead
0     26.763947          9     NaN     1
1     14.395988          7     NaN     1
2     13.784597          4     NaN     1
3      1.926665          7     NaN     1
4      9.743202         12     NaN     1
5      4.108450          9     8.0     1
